In [1]:
import open3d as o3d
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## load cad model

In [2]:
mesh_target = o3d.io.read_triangle_mesh('../dataset/model/art_melon.STL')
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0.0,0.0,0.0])
o3d.visualization.draw_geometries([mesh_target, mesh_frame])

In [3]:
# estimate normal
mesh_target.compute_triangle_normals()
mesh_target.orient_triangles()
pcd_target = mesh_target.sample_points_uniformly(number_of_points=10000)

In [4]:
o3d.visualization.draw_geometries([pcd_target])

## load point cloud

In [5]:
pcd_source = o3d.io.read_point_cloud('../dataset/d405/d405-1.ply')
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0.0,0.0,0.0])
o3d.visualization.draw_geometries([pcd_source, mesh_frame])

In [6]:
# estimate normal
pcd_source.orient_normals_consistent_tangent_plane(k=15)
o3d.visualization.draw_geometries([pcd_source, mesh_frame])

## downsampling

In [7]:
voxel_size = 0.005
pcd_source_downsample = pcd_source.voxel_down_sample(voxel_size * 1.5)
pcd_target_downsample = pcd_target.voxel_down_sample(voxel_size * 1.5)

In [8]:
pcd_source_downsample.estimate_normals()
pcd_target_downsample.estimate_normals()

In [9]:
o3d.visualization.draw_geometries([pcd_source_downsample])

In [10]:
o3d.visualization.draw_geometries([pcd_target_downsample])

## fpfh feature extraction

In [11]:
fpfh_source = o3d.pipelines.registration.compute_fpfh_feature(
    pcd_source_downsample, o3d.geometry.KDTreeSearchParamHybrid(radius=5.0*voxel_size, max_nn=30))
fpfh_target = o3d.pipelines.registration.compute_fpfh_feature(
    pcd_target_downsample, o3d.geometry.KDTreeSearchParamHybrid(radius=5.0*voxel_size, max_nn=30))

In [12]:
def visualize_feature_correspondences(source, target, source_fpfh, target_fpfh, num_display=500):
    # 1. KDTreeでtarget側から最も近い特徴点を探す
    search_tree = o3d.geometry.KDTreeFlann(target_fpfh)
    
    points = []
    lines = []
    
    # 表示する点数を制限（多すぎると見づらいため）
    indices = np.random.randint(0, len(source.points), num_display)
    
    for i in indices:
        # sourceのi番目の特徴量に最も近いtargetのインデックスを1つ探す
        [k, idx, dist] = search_tree.search_knn_vector_xd(source_fpfh.data[:, i], 1)
        
        if k > 0:
            # 線の始点（source）と終点（target）の座標
            points.append(source.points[i])
            points.append(target.points[idx[0]])
            # 線を定義（2*n 番目と 2*n+1 番目の結びつき）
            curr_idx = len(lines)
            lines.append([2*curr_idx, 2*curr_idx + 1])

    # 2. LineSetオブジェクトの作成
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(np.array(points))
    line_set.lines = o3d.utility.Vector2iVector(np.array(lines))
    # 線を青色にする
    line_set.paint_uniform_color([0, 0, 1])

    # 3. 可視化（sourceを少し横にずらすと見やすい）
    source_temp = copy.deepcopy(source).translate([0, 0, 0]) # 適宜調整
    line_set.translate([0, 0, 0]) 
    
    o3d.visualization.draw_geometries([source_temp, target, line_set])

# 実行例
visualize_feature_correspondences(pcd_source_downsample, pcd_target_downsample, fpfh_source, fpfh_target)

## ransac

In [13]:
distance_threshold = voxel_size * 1.5
result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    pcd_source_downsample, pcd_target_downsample, fpfh_source, fpfh_target, True, distance_threshold,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
    ransac_n=3,
    checkers = [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold),
               ],
    criteria = o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)
)

[Open3D WARNING] Too few correspondences (574) after mutual filter, fall back to original correspondences.


In [14]:
result

RegistrationResult with fitness=0.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 0
Access transformation to get result.